# fkpt: obtain LS values for functions A, Ap, KR1, KR1p

EdS values are A_LS, Ap_LS, KR1_LS, KR1p_LS = 1,0,1,1.   

LCDM values are close to EdS. For MG theories that reduce to LCDM at large scales, the values are the same as in LCDM 


EdS conditions are assumed at early times

In [2]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

# Linear growth equation is

$D''(k,\eta) + (2 + H'/H) D'(k,\eta) - \frac{3}{2} \Omega_m(\eta) \mu(k,\eta) D(\eta) =0$ 

with $\eta=\log(a)$


In [4]:


## Change mu for your MG
def mu(eta, k, OmegaM):
    beta2 = 1.0 / 6.0
    nHS = 1.0
    invH0 = 2997.92458

    mass = (
        (1.0 / invH0)
        * np.sqrt(0.5)
        * (OmegaM * np.exp(-3.0 * eta) + 4.0 * (1.0 - OmegaM)) ** ((2.0 + nHS) / 2.0)
        / (OmegaM + 4.0 * (1.0 - OmegaM)) ** ((1.0 + nHS) / 2.0)
    )

    return 1.0 + (2.0 * beta2 * k**2) / (k**2 + np.exp(2.0 * eta) * mass**2)

#Change f1 and f2 is your background is not LCDM:
 
# f2 =  (2 + H'/H)
def f2(eta, OmegaM):
    return 2.0 - 3.0 / (2.0 * (1.0 + (1.0 - OmegaM) / OmegaM * np.exp(3.0 * eta)))

# f1 =  3/2 OmegaM(eta)
def f1(eta, OmegaM):
    return 3.0 / (2.0 * (1.0 + (1.0 - OmegaM) / OmegaM * np.exp(3.0 * eta)))




In [5]:
def fkpt_LS_values(z,OmM,f0,
    mu_func,
    f1_func,
    f2_func
):
    
    if z<0.00001*1.01:
        z=0.00001*1.01
    output = compute_LSvalues(
        etaev=np.log(1/(1+z)),
        xev=0.0,kev=1e-8,pev=1e-8,
        f0ev=f0,OmegaM=OmM,
        mu_func=mu_func,
        f1_func=f1_func,
        f2_func=f2_func,
    )
    return output






def compute_LSvalues(
    etaev,
    xev,
    kev,
    pev,
    f0ev,
    OmegaM,
    mu_func,
    f2_func,
    f1_func,
    *,
    etainI=-4.0,
    etafinal=0.0,
    rtol=1e-6,
    atol=1e-9,
):
    import numpy as np
    from scipy.integrate import solve_ivp
    from scipy.interpolate import interp1d

    # --------------------------------------------------
    # Geometry
    # --------------------------------------------------

    def kplusp(x, k, p):
        return np.sqrt(k*k + p*p + 2.0*k*p*x)

    # --------------------------------------------------
    # Source kernels
    # --------------------------------------------------

    def source2a(eta, x, k, p):
        return f1_func(eta, OmegaM) * mu_func(eta, kplusp(x, k, p), OmegaM)

    def source2b(eta, x, k, p):
        return f1_func(eta, OmegaM) * (
            mu_func(eta, k, OmegaM)
            + mu_func(eta, p, OmegaM)
            - mu_func(eta, kplusp(x, k, p), OmegaM)
        )

    def source2FL(eta, x, k, p):
        kp = kplusp(x, k, p)
        return f1_func(eta, OmegaM) * (
            mu_func(eta, kp, OmegaM) * (p/k + k/p) * x
            - k*x/p * mu_func(eta, k, OmegaM)
            - p*x/k * mu_func(eta, p, OmegaM)
        )

    def sourceD2(eta, x, k, p):
        return (
            source2a(eta, x, k, p)
            - source2b(eta, x, k, p)*x*x
            + source2FL(eta, x, k, p)
        )

    # --------------------------------------------------
    # ODE system
    # --------------------------------------------------

    def rhs(eta, y):
        (
            D3, dD3,
            D2p, dD2p,
            D2m, dD2m,
            Dpk, dDpk,
            Dpp, dDpp
        ) = y

        S3I = (
            (f1_func(eta, OmegaM) *
             (mu_func(eta, pev, OmegaM)
              + mu_func(eta, kplusp(xev, kev, pev), OmegaM)
              - mu_func(eta, kev, OmegaM))
             * D2p * Dpp
             + sourceD2(eta, xev, kev, pev) * Dpk * Dpp * Dpp)
            * (1.0 - xev**2)
            / (1.0 + (pev/kev)**2 + 2.0*pev/kev*xev)
        )

        S3I += (
            (f1_func(eta, OmegaM) *
             (mu_func(eta, pev, OmegaM)
              + mu_func(eta, kplusp(-xev, kev, pev), OmegaM)
              - mu_func(eta, kev, OmegaM))
             * D2m * Dpp
             + sourceD2(eta, -xev, kev, pev) * Dpk * Dpp * Dpp)
            * (1.0 - xev**2)
            / (1.0 + (pev/kev)**2 - 2.0*pev/kev*xev)
        )

        S3II = (
            -f1_func(eta, OmegaM) *
            (mu_func(eta, pev, OmegaM)
             + mu_func(eta, kplusp(xev, kev, pev), OmegaM)
             - 2.0*mu_func(eta, kev, OmegaM))
            * Dpp * (D2p + Dpk*Dpp*xev**2)
            - f1_func(eta, OmegaM) *
            (mu_func(eta, kplusp(xev, kev, pev), OmegaM)
             - mu_func(eta, kev, OmegaM))
            * Dpk * Dpp * Dpp
        )
        S3II *= 2.0

        return np.array([
            dD3,
            -f2_func(eta, OmegaM)*dD3
            + f1_func(eta, OmegaM)*mu_func(eta, kev, OmegaM)*D3
            + S3I + S3II,

            dD2p,
            -f2_func(eta, OmegaM)*dD2p
            + f1_func(eta, OmegaM)*mu_func(eta, kplusp(xev, kev, pev), OmegaM)*D2p
            + sourceD2(eta, xev, kev, pev)*Dpk*Dpp,

            dD2m,
            -f2_func(eta, OmegaM)*dD2m
            + f1_func(eta, OmegaM)*mu_func(eta, kplusp(-xev, kev, pev), OmegaM)*D2m
            + sourceD2(eta, -xev, kev, pev)*Dpk*Dpp,

            dDpk,
            -f2_func(eta, OmegaM)*dDpk
            + f1_func(eta, OmegaM)*mu_func(eta, kev, OmegaM)*Dpk,

            dDpp,
            -f2_func(eta, OmegaM)*dDpp
            + f1_func(eta, OmegaM)*mu_func(eta, pev, OmegaM)*Dpp
        ])

    # --------------------------------------------------
    # Initial conditions
    # --------------------------------------------------

    Dplusi = np.exp(etainI)
    D2plusi = 3.0*np.exp(2.0*etainI)/7.0*(1.0-xev**2)

    D3symmi = (
        5.0/7.0*np.exp(3.0*etainI)/9.0 *
        (
            (1.0-xev**2)/(1.0+(pev/kev)**2+2.0*pev/kev*xev)
            + (1.0-xev**2)/(1.0+(pev/kev)**2-2.0*pev/kev*xev)
        )*(1.0-xev**2)
    )

    y0 = np.array([
        D3symmi, 3.0*D3symmi,
        D2plusi, 2.0*D2plusi,
        D2plusi, 2.0*D2plusi,
        Dplusi,  Dplusi,
        Dplusi,  Dplusi
    ])

    # --------------------------------------------------
    # Solve
    # --------------------------------------------------

    sol = solve_ivp(rhs, (etainI, etafinal), y0, rtol=rtol, atol=atol)
    interp = interp1d(sol.t, sol.y, axis=1, kind="cubic")

    Y  = interp(etaev)
    Yp = interp(etaev + 1e-5)
    Ym = interp(etaev - 1e-5)

    D3, dD3, D2p, _, _, _, Dpk, _, Dpp, _ = Y

    # --------------------------------------------------
    # Observables
    # --------------------------------------------------

    denom = (3.0/7.0)*Dpk*Dpp

    ALS = D2p / denom

    AprimeLS = (
        (Yp[2]/((3.0/7.0)*Yp[6]*Yp[8])
         - Ym[2]/((3.0/7.0)*Ym[6]*Ym[8]))
        / (2e-5)
    )

    KR1LS  = D3  / (Dpk*Dpp*Dpp) * 21.0/5.0
    KR1pLS = dD3 / (Dpk*Dpp*Dpp) * 21.0/5.0 / (3.0*f0ev)

    return np.array([ALS, AprimeLS, KR1LS, KR1pLS])




In [15]:
# WARNING: A change on z or OmM implies a change on f0. Hence, be careful, or you will obtain non-sense KR1p_LS values
fkpt_LS_values(z=0.5,OmM=0.3,f0=0.749238,
    mu_func=mu,
    f1_func=f1,
    f2_func=f2
)

array([1.00374171, 0.00885461, 1.0069998 , 1.01630093])